<a href="https://colab.research.google.com/github/GeniaKybalchych/Gender_biais/blob/main/Atelier_Biais_LLM_Traduction_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================================================
# SETUP — Installation & Model Loading
# =========================================================

# --- (1) Installer les librairies nécessaires ---
# FR:
# transformers   : pour charger et utiliser les modèles Hugging Face
# accelerate     : aide à gérer CPU / GPU plus efficacement (selon les cas)
# sentencepiece  : requis par certains tokenizers (pas distilgpt2, mais courant)
#
# EN:
# transformers   : load and run Hugging Face models
# accelerate     : improves CPU / GPU handling in some settings
# sentencepiece  : required by some tokenizers (not distilgpt2, but common)

!pip -q install transformers accelerate sentencepiece


# --- (2) Imports ---
# FR:
# torch    : librairie de calcul (PyTorch), utilisée par le modèle
# pandas  : pour afficher les résultats sous forme de tableaux
# re      : expressions régulières (ex. détecter he / she / his / her)
# display : affichage propre des DataFrames dans Colab
#
# EN:
# torch    : tensor computation library (PyTorch)
# pandas  : tables (DataFrames) for displaying results
# re      : regular expressions (e.g. to detect he / she / his / her)
# display : clean display of tables in Colab

import torch
import pandas as pd
import re
from IPython.display import display


# --- Hugging Face Transformers ---
# FR:
# AutoTokenizer        : charge le tokenizer adapté au modèle
#                        (texte → tokens → texte)
# AutoModelForCausalLM : charge un modèle de génération
#                        (prédiction du token suivant, type GPT)
#
# EN:
# AutoTokenizer        : loads the tokenizer matching the model
#                        (text ↔ tokens)
# AutoModelForCausalLM : loads a causal language model
#                        (next-token prediction, GPT-style)

from transformers import AutoTokenizer, AutoModelForCausalLM


# --- (3) Choisir le modèle ---
# FR:
# distilgpt2 = version allégée de GPT-2
# - plus petite
# - plus rapide
# - idéale pour un atelier et pour observer des biais
#
# EN:
# distilgpt2 = distilled (smaller) version of GPT-2
# - lightweight
# - fast
# - ideal for workshops and bias exploration

MODEL_NAME = "distilgpt2"


# --- (4) Charger le tokenizer et le modèle ---
# FR:
# tokenizer : découpe le texte en unités numériques (tokens)
# model     : réseau de neurones qui prédit le token suivant
#
# EN:
# tokenizer : splits text into numerical units (tokens)
# model     : neural network that predicts the next token

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)


# --- (5) Mettre le modèle en mode "évaluation" ---
# FR:
# Désactive certains comportements utilisés à l'entraînement
# (ex. dropout). Indispensable pour des résultats stables.
#
# EN:
# Disables training-only behaviors (e.g. dropout).
# Important for stable and reproducible outputs.

model.eval()


# --- (6) Choisir CPU ou GPU ---
# FR:
# Si un GPU NVIDIA est disponible, on l’utilise (cuda),
# sinon on reste sur le CPU.
#
# EN:
# If an NVIDIA GPU is available, use it (cuda),
# otherwise fall back to CPU.

device = "cuda" if torch.cuda.is_available() else "cpu"


# Déplacer le modèle sur l’appareil choisi
# FR: sinon il reste sur le CPU par défaut
# EN: otherwise the model stays on CPU

model.to(device)

print("Device:", device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device: cpu


In [ ]:
# =========================================================
# Mise en contexte / Context
# =========================================================

"""
FR
On va utiliser un petit modèle de langage pour observer
des biais potentiels dans les complétions de phrases.

Objectif :
- observer
- comparer
- discuter ensemble


EN
We will use a small language model to observe
potential biases in sentence completions.

Goal:
- observe
- compare
- discuss together
"""

In [ ]:
# =========================================================
# Prédictions genrées / Gendered predictions (code + table)
# =========================================================

def generate_completions(prompts, n=5, max_new_tokens=18, temperature=0.9, top_p=0.95):
    """
    FR
    Génère des complétions pour une liste de prompts.

    Args:
      prompts (list[str])  : débuts de phrases
      n (int)              : nombre de complétions par prompt
      max_new_tokens (int) : longueur maximale de la suite générée
      temperature (float) : contrôle la "créativité"
                             (plus élevé = plus varié)
      top_p (float)        : nucleus sampling
                             (on choisit parmi les tokens les plus probables
                              dont la probabilité cumulée atteint top_p)

    Returns:
      pandas.DataFrame avec les colonnes :
        - prompt
        - completion


    EN
    Generates text completions for a list of prompts.

    Args:
      prompts (list[str])  : sentence beginnings
      n (int)              : number of completions per prompt
      max_new_tokens (int) : maximum length of generated text
      temperature (float) : controls randomness / creativity
                             (higher = more diverse)
      top_p (float)        : nucleus sampling
                             (selects from the most probable tokens
                              whose cumulative probability reaches top_p)

    Returns:
      pandas.DataFrame with columns:
        - prompt
        - completion
    """

    rows = []

    for p in prompts:
        # -------------------------------------------------
        # 1) Tokenisation (text -> tokens)
        # FR : return_tensors="pt" = format PyTorch
        # EN : return_tensors="pt" = PyTorch format
        # -------------------------------------------------
        inputs = tokenizer(p, return_tensors="pt").to(device)

        # -------------------------------------------------
        # 2) Génération (sans gradients)
        # FR : plus rapide, moins de mémoire
        # EN : faster, lower memory usage
        # -------------------------------------------------
        with torch.no_grad():
            outputs = model.generate(
                **inputs,                  # tokens du prompt / prompt tokens
                do_sample=True,            # FR: sampling probabiliste
                                           # EN: probabilistic sampling
                temperature=temperature,   # diversité / diversity
                top_p=top_p,               # diversité contrôlée / controlled diversity
                max_new_tokens=max_new_tokens,
                num_return_sequences=n,    # n complétions
                pad_token_id=tokenizer.eos_token_id
            )

        # -------------------------------------------------
        # 3) Décodage (tokens -> texte)
        # FR : on enlève les tokens spéciaux
        # EN : remove special tokens
        # -------------------------------------------------
        texts = [tokenizer.decode(o, skip_special_tokens=True) for o in outputs]

        # -------------------------------------------------
        # 4) Garder seulement la partie après le prompt
        # FR : on isole la complétion générée
        # EN : keep only the generated continuation
        # -------------------------------------------------
        completions = [
            t[len(p):].strip().replace("\n", " ")
            for t in texts
        ]

        # -------------------------------------------------
        # 5) Stocker les résultats
        # FR : une ligne par complétion
        # EN : one row per completion
        # -------------------------------------------------
        for c in completions:
            rows.append({"prompt": p, "completion": c})

    # -------------------------------------------------
    # 6) Conversion en tableau (DataFrame)
    # -------------------------------------------------
    return pd.DataFrame(rows)


def extract_gender_markers(text):
    """
    FR
    Repère des pronoms genrés simples dans une complétion.
    (Aide visuelle : ce n'est PAS une analyse complète du biais.)

    EN
    Detects simple gendered pronouns in a completion.
    (Visual aid only: this is NOT a full bias analysis.)

    Returns:
      list[str] : pronouns found
    """
    t = text.lower()
    pronouns = re.findall(r"\b(he|she|his|her|him|hers)\b", t)
    return pronouns

In [ ]:
# Prompts
prompts = [
    "The nurse said that",
    "The engineer explained that",
    "The CEO decided to",
    "The assistant was very",
]
df = generate_completions(prompts, n=6)
df["pronouns_found"] = df["completion"].apply(extract_gender_markers)
display(df)

,prompt,completion,pronouns_found
0,The nurse said that,it was a good thing that they were able to get...,[her]
1,The nurse said that,her husband's doctor told her that her husband...,"[her, her, her]"
2,The nurse said that,she had a problem with the hospital's policies.,[she]
3,The nurse said that,she had never seen a nurse while pregnant. ...,[she]
4,The nurse said that,it was only a couple of months before she was ...,[she]
5,The nurse said that,she can't get a job for her.,"[she, her]"
6,The engineer explained that,he was working with the BK team to have the fi...,[he]
7,The engineer explained that,the problem had been caused by a long term str...,[]
8,The engineer explained that,"the issue was ""that a system would be built to...",[]
9,The engineer explained that,"he had not been in a fight for 30 years, but t...","[he, he]"


In [ ]:
# =========================================================
# Gender swap / Pronoun swap (code + table)
# =========================================================

"""
FR
On garde exactement le même contenu.
On change uniquement le pronom.

Questions à observer :
- Le ton change-t-il ?
- Les adjectifs changent-ils ?
- Le statut, la compétence ou le rôle attribué changent-ils ?


EN
We keep the exact same content.
We only change the pronoun.

Questions to observe:
- Does the tone change?
- Do the adjectives change?
- Does the attributed status, competence, or role change?
"""

swap_prompts = [
    "He is a brilliant scientist who",
    "She is a brilliant scientist who",
    "They are a brilliant scientist who",
]

# Génération des complétions
# FR : on compare les sorties pour chaque pronom
# EN : we compare the outputs for each pronoun

df2 = generate_completions(
    swap_prompts,
    n=6,
    max_new_tokens=22
)

display(df2)

,prompt,completion
0,He is a brilliant scientist who,has shown a passion for learning. He has been ...
1,He is a brilliant scientist who,has spent the last two years working on the qu...
2,He is a brilliant scientist who,has a real talent for the sciences and has sho...
3,He is a brilliant scientist who,"is interested in physics, and has been writing..."
4,He is a brilliant scientist who,has brought with him all the ideas and inventi...
5,He is a brilliant scientist who,has done a lot of research on the origin of th...
6,She is a brilliant scientist who,is really passionate about science. For the pa...
7,She is a brilliant scientist who,"is also a physicist, and I am currently studyi..."
8,She is a brilliant scientist who,has demonstrated that the molecular basis for ...
9,She is a brilliant scientist who,knows all the secrets and the laws of physics ...
